In [12]:
import sqlite3
conn = sqlite3.connect(':memory:')

# Create tables
create_products_table = '''
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    item_name VARCHAR(255),
    item_category VARCHAR(255),
    item_size VARCHAR(50),
    price DECIMAL(10, 2)
);
'''

create_customers_table = '''
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    delivery_address VARCHAR(255)
);
'''

create_orders_table = '''
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE
);
'''

create_order_details_table = '''
CREATE TABLE order_details (
    row_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    item_quantity INT,
    item_price DECIMAL(10, 2)
);
'''

conn.executescript(create_products_table)
conn.executescript(create_customers_table)
conn.executescript(create_orders_table)
conn.executescript(create_order_details_table)

# Populate Data
insert_products = '''
INSERT INTO products (product_id, item_name, item_category, item_size, price)
VALUES
    (4, 'Hawaiian', 'Classic', 'Large', 15.99),
    (5, 'Meat Feast', 'Meat Lovers', 'Medium', 12.99),
    (6, 'Margarita', 'Classic', 'Small', 9.99),
    (7, 'Vegetarian Deluxe', 'Vegetarian', 'Large', 16.99),
    (8, 'BBQ Chicken', 'Specialty', 'Medium', 14.99);
'''

insert_customers = '''
INSERT INTO customers (customer_id, customer_name, delivery_address)
VALUES
    (3, 'Alice Johnson', '789 Pine St, Villageton'),
    (4, 'Bob Williams', '101 Cedar St, Townsville');
'''

insert_orders = '''
INSERT INTO orders (order_id, customer_id, order_date)
VALUES
    (3, 3, '2023-01-05'),
    (4, 4, '2023-01-07');

'''

insert_order_details = '''
INSERT INTO order_details (row_id, order_id, product_id, item_quantity, item_price)
VALUES
    (4, 3, 4, 1, 15.99),
    (5, 3, 5, 2, 25.98),
    (6, 4, 6, 1, 9.99),
    (7, 4, 7, 3, 50.97),
    (8, 4, 8, 2, 29.98);
'''

conn.executescript(insert_products)
conn.executescript(insert_customers)
conn.executescript(insert_orders)
conn.executescript(insert_order_details)

conn.commit()

# Query 1: Retrieve Customer Orders with Pizza Details
query1 = '''
SELECT
    c.customer_name,
    o.order_id,
    o.order_date,
    p.item_name,
    od.item_quantity,
    od.item_price
FROM
    customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
JOIN products p ON od.product_id = p.product_id;
'''

# Query 2: Top Selling Pizzas
query2 = '''
SELECT
    p.item_name,
    SUM(od.item_quantity) as total_sold
FROM
    products p
JOIN order_details od ON p.product_id = od.product_id
GROUP BY
    p.item_name
ORDER BY
    total_sold DESC;
'''

# Query 3: Total Revenue for Each Customer
query3 = '''
SELECT
    c.customer_name,
    SUM(od.item_price) as total_spent
FROM
    customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
GROUP BY
    c.customer_name
ORDER BY
    total_spent DESC;
'''

# Query 4: Busiest Day of the Week
query4 = '''
SELECT
    strftime('%Y-%m-%d', o.order_date) as order_date,
    COUNT(o.order_id) as order_count
FROM
    orders o
GROUP BY
    order_date
ORDER BY
    order_count DESC
LIMIT 1;
'''

# Query 5: Customer Retention Rate
query5 = '''
SELECT
    o1.order_date,
    COUNT(DISTINCT o2.order_id) as returning_customers
FROM
    orders o1
LEFT JOIN orders o2 ON o1.customer_id = o2.customer_id AND o1.order_date > o2.order_date
GROUP BY
    o1.order_date
ORDER BY
    o1.order_date;

'''

result1 = conn.execute(query1).fetchall()
result2 = conn.execute(query2).fetchall()
result3 = conn.execute(query3).fetchall()
result4 = conn.execute(query4).fetchall()
result5 = conn.execute(query5).fetchall()


print("Query 1: Retrieve Customer Orders with Pizza Details")
print(result1)

print("\nQuery 2: Top Selling Pizzas")
print(result2)

print("\nQuery 3: Total Revenue for Each Customer")
print(result3)

print("\nQuery 3: Busiest Day of the Week")
print(result4)

print("\nQuery 5: Customer Retention Rate")
print(result5)

conn.close()


Query 1: Retrieve Customer Orders with Pizza Details
[('Alice Johnson', 3, '2023-01-05', 'Hawaiian', 1, 15.99), ('Alice Johnson', 3, '2023-01-05', 'Meat Feast', 2, 25.98), ('Bob Williams', 4, '2023-01-07', 'Margarita', 1, 9.99), ('Bob Williams', 4, '2023-01-07', 'Vegetarian Deluxe', 3, 50.97), ('Bob Williams', 4, '2023-01-07', 'BBQ Chicken', 2, 29.98)]

Query 2: Top Selling Pizzas
[('Vegetarian Deluxe', 3), ('Meat Feast', 2), ('BBQ Chicken', 2), ('Margarita', 1), ('Hawaiian', 1)]

Query 3: Total Revenue for Each Customer
[('Bob Williams', 90.94), ('Alice Johnson', 41.97)]

Query 3: Busiest Day of the Week
[('2023-01-07', 1)]

Query 5: Customer Retention Rate
[('2023-01-05', 0), ('2023-01-07', 0)]
